# Introduction to GAP in Julia - Exercises

In [1]:
using GAP

Adding path /home/sebastian/Software/GAPJulia/gap/.libs to DL_LOAD_PATH
 ┌───────┐   GAP 4.dev of today
 │  GAP  │   https://www.gap-system.org
 └───────┘   Architecture: x86_64-pc-linux-gnu-julia64-kv6
 Configuration:  gmp 6.1.2, Julia GC, Julia 1.1.0, readline
 Loading the library and packages ...
 Packages:   GAPDoc 1.6.1, PrimGrp 3.3.1, SmallGrp 1.3, TransGrp 2.0.4
 Try '??help' for help. See also '?copyright', '?cite' and '?authors'


In this exercise sheet, we will explore how to work with GAP groups from Julia

## Exercise 1: Computing Orbits

We want to compute the orbits of $S_5$ operating on tuples of integers smaller or equal to 5 of length 4. Use the tuple operation and the GAP `Orbits` command.

First, we prepare the input

In [2]:
action(tuple,perm) = Tuple( i^perm for i in tuple )
len = 5
grp = GAP.Globals.SymmetricGroup(len)
input = reshape( [ tuple(i,j,k,l) for i=1:len, j=1:len, k=1:len, l=1:len ], len^4 );
gap_input = GAP.julia_to_gap( input );

Now, we compute the orbits and look at the output

In [3]:
gap_orbs = GAP.Globals.Orbits(grp,gap_input,GAP.julia_to_gap(action));

In [4]:
length(gap_orbs)

15

In [5]:
[ gap_orbs[i][1] for i in 1:length(gap_orbs) ]

15-element Array{NTuple{4,Int64},1}:
 (1, 1, 1, 1)
 (2, 1, 1, 1)
 (1, 2, 1, 1)
 (2, 2, 1, 1)
 (3, 2, 1, 1)
 (1, 1, 2, 1)
 (2, 1, 2, 1)
 (3, 1, 2, 1)
 (1, 2, 2, 1)
 (2, 2, 2, 1)
 (3, 2, 2, 1)
 (1, 3, 2, 1)
 (2, 3, 2, 1)
 (3, 3, 2, 1)
 (4, 3, 2, 1)

## Exercise 2: Operating on matrices

Write a function `oper(mat::Array{Int64,2},perm::GAP.MPtr)` that operates on the matrix `mat` by permuting the columns

In [6]:
function oper(mat::Array{Int64,2},perm::GAP.MPtr)
    new_mat = copy(mat)
    for i in 1:size(mat)[2]
        new_mat[:,i^perm] = mat[:,i]
    end
    return new_mat
end

oper (generic function with 1 method)

Please test your output using the following data

In [7]:
perm = GAP.Globals.PermList(GAP.julia_to_gap([2,3,4,1]))
mat = [ 1 2 3 4 ; 4 3 2 1 ]
oper(mat,perm) == [ 4 1 2 3 ; 1 4 3 2]

true

## Exercise 3: Operating on matrices

Use the function `oper` to compute the orbit of the matrix `[1 2 1 2; 1 3 1 4]` under $S_4$, $A_4$, and $C_4$

In [8]:
mat = [1 2 1 2; 1 3 1 4]

2×4 Array{Int64,2}:
 1  2  1  2
 1  3  1  4

In [9]:
grp = GAP.Globals.SymmetricGroup(4)
orbit = GAP.Globals.Orbit(grp,mat,GAP.julia_to_gap(oper))

GAP: [ [1 2 1 2; 1 3 1 4], [2 1 2 1; 4 1 3 1], [2 1 1 2; 3 1 1 4], [1 2 1 2; 1 4 1 3], [1 2 2 1; 1 4 3 1], [2 2 1 1; 4 3 1 1], [2 1 2 1; 3 1 4 1], [2 1 1 2; 4 1 1 3], [1 1 2 2; 1 1 4 3], [2 2 1 1; 3 4 1 1], [1 2 2 1; 1 3 4 1], [1 1 2 2; 1 1 3 4] ]

In [10]:
grp = GAP.Globals.AlternatingGroup(4)
orbit = GAP.Globals.Orbit(grp,mat,GAP.julia_to_gap(oper))

GAP: [ [1 2 1 2; 1 3 1 4], [1 1 2 2; 1 1 3 4], [1 2 2 1; 1 4 3 1], [2 1 1 2; 3 1 1 4], [1 2 1 2; 1 4 1 3], [2 1 2 1; 3 1 4 1], [1 1 2 2; 1 1 4 3], [2 2 1 1; 3 4 1 1], [1 2 2 1; 1 3 4 1], [2 2 1 1; 4 3 1 1], [2 1 1 2; 4 1 1 3], [2 1 2 1; 4 1 3 1] ]

In [18]:
grp = GAP.Globals.Group(GAP.Globals.PermList(GAP.julia_to_gap([2,3,4,1])))
orbit = GAP.Globals.Orbit(grp,mat,GAP.julia_to_gap(oper))

GAP: [ [1 2 1 2; 1 3 1 4], [2 1 2 1; 4 1 3 1], [1 2 1 2; 1 4 1 3], [2 1 2 1; 3 1 4 1] ]

## Exercise 4: Stabilizer Chain

Write a function that, given a group $G$, a list of elements $l$, and an action $\phi$, computes the stabilizer chain of that operation, i.e., a list
$$G_{l_1} \geq G_{l_1,l_2} \geq ...$$

In [11]:
function stab_chain(grp, list, action)
    stabs = []
    current_stab = grp
    for i in list
        current_stab = GAP.Globals.Stabilizer(current_stab,i,GAP.gap_to_julia(action))
        push!(stabs, current_stab)
    end
    return stabs
end

stab_chain (generic function with 1 method)

Test your function with the following input

In [12]:
oper2(i,perm) = i^perm
grp = GAP.Globals.SymmetricGroup(5)
list = [1,2,3,4,5]

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

In [13]:
stab_chain(grp,list,oper2)

5-element Array{Any,1}:
 GAP: SymmetricGroup( [ 2 .. 5 ] )
 GAP: SymmetricGroup( [ 3 .. 5 ] )
 GAP: SymmetricGroup( [ 4 .. 5 ] )
 GAP: Group( () )                 
 GAP: Group( () )                 